# Rhythm Interpolation Methods 

In [ ]:
import os
import sys
print(sys.version)

In [ ]:
import warnings

import joblib

import numpy as np
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

In [ ]:
sys.path.append('./COMMON_UTILS/')

In [ ]:
from wasserstein_transformations import SmoothTransform

In [ ]:
warnings.simplefilter("ignore")
plt.rcParams['figure.figsize'] = (15, 5)

### Threshold Transformation 

In [ ]:
src, dst = joblib.load('./activations.pkl')

In [ ]:
plt.plot(src[0, :])
plt.plot(dst[0, :]);

In [ ]:
t = np.linspace(0, 1, 50)
src_t = np.power(1-t, 2)
dst_t = np.power(t, 2)

plt.plot(src_t)
plt.plot(dst_t)
plt.show()

#### Activation resynthesis 

In [ ]:
act = src[2, :200]
act /= act.max()
peaks, peaks_info = find_peaks(np.insert(act, 0, 0), distance=2, prominence=0.05)
peaks -= 1

In [ ]:
from findpeaks import findpeaks

In [ ]:
fp = findpeaks(lookahead=3, method='topology', verbose=0)

In [ ]:
data = fp.fit(np.insert(act, 0, 0))

In [ ]:
data['df']['labx'] = data['df']['labx'].astype(int)

In [ ]:
groups = data['df'].groupby("labx")

In [ ]:
data['df']

In [ ]:
peak_heights = groups['y'].max()
peak_score = groups['score'].max()
peak_idxs = groups.indices
peak_lengths = dict([(i, len(a)) for i, a in peak_idxs.items()])

In [ ]:
candidate_peaks = np.where(peak_heights > peak_heights.max()/2)
candidate_peaks = 

In [ ]:
peak_idxs

In [ ]:
fp.plot()

In [ ]:
act_r = np.zeros_like(act)

t = 0.8

for i in peak_idxs.keys():
    if peak_score[i] < 0.2:
        continue
    
    if peak_heights[i] < t:
        continue
        
    act_r[peak_idxs[i]-1] = act[peak_idxs[i]-1]

In [ ]:
plt.plot(act_r)

In [ ]:
peaks_info

In [ ]:
plt.plot(act, marker='.')
plt.scatter(peaks, act[peaks])
for i in range(len(peaks_info['left_bases'])):
    left = peaks_info['left_bases'][i]
    right = peaks_info['right_bases'][i]
#     plt.axvspan(left, right, alpha=0.3)
    plt.plot([left, right], [act[peaks[i]]]*2,)

In [ ]:
def makePeak(height):
    atk = np.power(np.linspace(0, 1, 3), 6) * height
    dec = np.power(np.linspace(1, 0, 4), 6) * height
    
    peak = np.concatenate([atk, dec[1:]])
    return peak, len(atk) - 1

In [ ]:
act_h = np.zeros_like(act)

for p in peaks:
    h = act[p]
    peak_h, idx_offset = makePeak(h)
    
    start = p - idx_offset
    if start < 0:
        peak_h = peak_h[-start:]
        start = 0
        
    length = min(len(peak_h), len(act_h) - start)    
    end = start + length
    
    act_h[start:end] += peak_h[:length]
    
plt.plot(act)
plt.plot(act_h)
plt.show()

In [ ]:
class SmoothTransition:
    def __init__(self, src, trg):
        assert src.shape == trg.shape
        
        self.src = src / src.max()
        self.trg = trg / trg.max()
        
        self.src_peaks, _ = find_peaks(np.insert(self.src, 0, 0), distance=2, prominence=0.01)
        self.trg_peaks, _ = find_peaks(np.insert(self.trg, 0, 0), distance=2, prominence=0.01)
        
        self.src_peaks -= 1
        self.trg_peaks -= 1
    
    def transform(self, t):
        t = min(max(t, 0.0), 1.0)

        peaks_src = self.src_peaks[np.where(self.src[self.src_peaks] >= t)]
        peaks_trg = self.trg_peaks[np.where(self.trg[self.trg_peaks] <= t)]
        
        act_t = np.zeros_like(self.src)
        
        for ps in peaks_src:
            act_t = self.add_peak(act_t, ps, self.src[ps])
            
        for pt in peaks_trg:
            act_t = self.add_peak(act_t, pt, self.trg[pt])
            
        return act_t
    
    __call__ = transform
            
    @staticmethod
    def add_peak(a, p, h):
        peak_h, idx_offset = SmoothTransition.makePeak(h)

        start = p - idx_offset
        if start < 0:
            peak_h = peak_h[-start:]
            start = 0

        length = min(len(peak_h), len(a) - start)    
        end = start + length

        a[start:end] += peak_h[:length]
        return a
    
    @staticmethod
    def makePeak(height):
        atk = np.power(np.linspace(0, 1, 3), 6) * height
        dec = np.power(np.linspace(1, 0, 4), 6) * height

        peak = np.concatenate([atk, dec[1:]])
        return peak, len(atk) - 1

In [ ]:
sm = SmoothTransition(src[0], dst[0])

In [ ]:
act_h = sm(0.5)
plt.plot(src[1]/src[1].max())
plt.plot(dst[1]/dst[1].max())
plt.plot(act_h);

In [ ]:
plt.plot(sm.src)

offset = 1
for t in np.linspace(0, 1, 5):
    a = sm(t)
    plt.plot(a - offset*1.1, c='k')
    offset += 1
    
plt.plot(sm.trg - offset*1.1)
plt.show()

### Wasserstein Transformation 

In [ ]:
k = 300

ps, qs = src[0][:k], dst[0][:k]
ps, qs = ps/ps.sum(), qs/qs.sum()

In [ ]:
sm = SmoothTransform(steps=5)

In [ ]:
trans = sm.transform(ps, qs)

In [ ]:
sm.plot_transform(trans, source=ps, target=qs)

In [ ]:
trans.shape

### Custom Mehtod 

In [ ]:
v0 = np.zeros(12)
v0[1] = 0.5
v0[6] = 0.5

v1 = np.zeros(12)
v1[3] = 0.5
v1[9] = 0.5

In [ ]:
plt.plot(v0)
plt.plot(v1+0.01);

In [ ]:
T = np.zeros((12, 12))
T[1, 3] = 0.5
T[6, 9] = 0.5

In [ ]:
plt.matshow(T);

In [ ]:
def interp(v0, v1, T, p):
    vp = np.zeros_like(v0)
    for i in range(len(vp)):
        js = np.where(T[i] > 0)[0]
        for j in js:
            j_p = int(p * j + (1-p)*i)
            vp[j_p] += T[i, j]
            
    return vp

In [ ]:
for i, p in enumerate(np.linspace(0, 1, 5)):
    plt.plot(interp(v0, v1, T, p)+0.01*i, label=p)
    
plt.legend()
plt.show()